In [11]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Project1 import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "123"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################

app = JupyterDash('Grazioso Salvare Animal Shelter')

# Image location and encoding
image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Heading with image display and unique unique identifier
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center([
        html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),
             alt='customer image',style={'height': '15%', 'width': '15%'}
                ),
        html.H5('App Developed by Eugene Panchenko - SNHU - CS-340'),
    ]),
    html.Hr(),
    
# Interactive options to filter data (radioitems)
# https://dash.plotly.com/dash-core-components/radioitems
   html.Div(children=[
       dcc.RadioItems(id='rescueFilter',
                      options=[
                          {'label': 'Water Rescue', 'value': 'water'},
                          {'label': 'Mountain Rescue', 'value': 'mountain'},
                          {'label': 'Disaster Rescue', 'value': 'disaster'},
                          {'label': 'Reset', 'value': 'reset'}
                      ],value='reset', labelStyle={'display': 'inline-block'},
                      className="row",style={'display': 'flex'}
                     )
        ]),
    html.Hr(),     
    
# Data table with options to filter data by column or select row for additional options  
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that the pie chart and geolocation chart are side-by-side
    html.Div(children=[
        html.Div(dcc.Graph(id='pie'),
                 style={'display': 'inline-block'}),
        html.Div(id='map-id', className='col s12 m6'),
    ], style={'display' : 'flex'})
])

#############################################
# Interaction Between Components / Controller
#############################################

# Table Filtering using Radio Buttons
# Radio buttons regex hard coded by breeds, sex and age in weeks
# Refrence: https://www.regexpal.com/
@app.callback(
    Output("datatable-id", "data"),
    [Input("rescueFilter", "value")]
)
def  update_dashboard(filter_type):
    if filter_type == 'water':
        df = pd.DataFrame(list(shelter.read(
                {"$and": [{"$or": [
                    {"breed": {"$regex": ".*Labrador.*"}},
                    {"breed": {"$regex": ".*Chesapeake.*"}},
                    {"breed": {"$regex": ".*Newfoundland.*"}}
                ]},
                          {"sex_upon_outcome": {"$regex":"Intact Fem.*"}},
                          {"$and": [
                              {"age_upon_outcome_in_weeks": {"$gte": 26}},
                              {"age_upon_outcome_in_weeks": {"$lte": 156}}
                          ]}
                         ]}
        ))
                         )

    if filter_type == 'mountain':
        df = pd.DataFrame(list(shelter.read(
            {"$and": [{"$or": [
                {"breed": {"$regex": ".*German Shepherd.*"}},
                {"breed": {"$regex": ".*Alaskan Malamute.*"}},
                {"breed": {"$regex": ".*Old English Sheepdog.*"}},
                {"breed": {"$regex": ".*Siberian Husky.*"}},
                {"breed": {"$regex": ".*Rottweiler.*"}}]},
                      {"sex_upon_outcome": {"$regex": "Intact Male"}},
                      {"$and": [
                          {"age_upon_outcome_in_weeks": {"$gte": 26}},
                          {"age_upon_outcome_in_weeks": {"$lte": 156}}
                      ]}
                     ]}
        ))
                         )

    if filter_type == 'disaster':
        df = pd.DataFrame(list(shelter.read(
                {"$and": [{"$or": [
                    {"breed": {"$regex": ".*Doberman.*"}},
                    {"breed": {"$regex": ".*German Shepherd.*"}},
                    {"breed": {"$regex": ".*Golden Retriever.*"}},
                    {"breed": {"$regex": ".*Bloodhound.*"}},
                    {"breed": {"$regex": ".*Rottweiler.*"}},
                ]},
                          {"sex_upon_outcome": "Intact Male"},
                    {"$and": [
                        {"age_upon_outcome_in_weeks": {"$gte": 20}},
                        {"age_upon_outcome_in_weeks": {"$lte": 300}}
                    ]}
                         ]}
        ))
                         )

    elif filter_type == 'reset':
        df = pd.DataFrame(list(shelter.read({})))
        
    return df.to_dict('records')


# Code provided, couldn't figure out what it does.
#
#@app.callback(
#    Output('datatable-id', 'style_data_conditional'),
#    [Input('datatable-id', 'selected_columns')]
#)
#def update_styles(selected_columns):
#    return [{
#        'if': { 'column_id': i },
#        'background_color': '#D2F3FF'
#    } for i in selected_columns]


# Callback for PieChart filtered by breed
# Refrence: https://plotly.com/python/pie-charts/
@app.callback(
    Output("pie", "figure"),
    [Input("datatable-id", "data")]
)
def update_graphs(Data):
    dff = pd.DataFrame.from_dict(Data)
    fig = px.pie(dff, names='breed', title='Filter by Breed')
    
    return fig


# Callback for Map Section
# Refrence: Module6Milestone
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),  
     Input('datatable-id', "selected_rows")]          
)
def update_map(viewData,row):
    dff = pd.DataFrame.from_dict(viewData)
    
    # Using row as user selection pull Longitude and Latitude
    lat = float(dff.iloc[row, 14])
    long = float(dff.iloc[row, 15])
    
    # Map Formatting
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat,long], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat,long], children=[
                
                # ToolTip Settings (Display Animal Name)
                dl.Tooltip(dff.iloc[row,9]),
                
                # Popup options (Display basic animal information in a readable format)
                dl.Popup([
                    html.H3(dff.iloc[row,9]),
                    html.H4(dff.iloc[row,1] + ' old ' + dff.iloc[row,12] + ' ' + dff.iloc[row,3]),
                    html.H4(dff.iloc[row,5] + ' ' + dff.iloc[row,4]),
                    html.H4('Outcome: ' + dff.iloc[row,11] + ' ' + dff.iloc[row,10])
                ])
            ])
        ])
    ]


app